# Prognositc (trajectory prediction) performance of baseline ML models on ADNI2 longitudinal data

* **Objectives:** 
    1. Predict Trajectory classes based on structural features
    2. Predict Trajectory classes based on structural features + baseline clinical score + demographics


* **Input modalities:**
    1. Coritcal thickness (CT)
    2. Hippocampal subfield volumes (HC) 


* **Models:**
    1. Logistic regression (Lasso)
    2. SVM
    3. Random Forest
    
    
* **Code organization:**
    1. Stage 1: 
        1. Import raw data, and setup CV folds based on choice of input features 
        2. Save this experimental setup to disk
        
    2. Stage 2:
        1. Train and test ML models over k-fold setup (Parallelized implmentation)
        2. Plot and save performance results 

In [1]:
# Basic Imports
import numpy as np
import pandas as pd
from scipy import stats
import pickle
import re
import collections
import tables as tb
from math import isnan
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import KFold

%matplotlib inline
#plt.rcParams['figure.figsize'] = (15, 10)
#plt.style.use('ggplot')

/projects/nikhil/ADNI_prediction/code/conda_envs/adni-conda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)
/projects/nikhil/ADNI_prediction/code/conda_envs/adni-conda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
/projects/nikhil/ADNI_prediction/code/conda_envs/adni-conda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This modul

In [2]:
# Some Defs
def load_data(data_path, input_node, preproc):
    data = tb.open_file(data_path, 'r')
    X_raw = data.get_node('/' + input_node)[:]
    if preproc == 'scale':
        X = preprocessing.scale(X_raw)
    elif preproc == 'norm_max':
        X = preprocessing.normalize(X_raw, norm='max')
    elif preproc == 'norm_l2':
        X = preprocessing.normalize(X_raw, norm='l2')
    else:
        X = X_raw
    data.close()
    return X

def pickleIt(my_data,save_path):
    f = open(save_path, 'wb')
    pickle.dump(my_data, f)
    f.close()

#Outer Fold Computation (need the imports inside def if you want to parallelize!)
def computeOuterFold(train_X, train_y, valid_X, valid_y, model_clf, hyper_params, output_type, model_type):    
    import collections
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Lasso
    from sklearn.svm import SVR
    from sklearn.ensemble import RandomForestRegressor
    from sklearn import grid_search
    import datetime
    import time
    import collections
    from scipy.stats import mode
    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae
    from scipy import stats
    import numpy as np
    
    print 'Starting Outerfold computation'
    
    hp_dict = collections.defaultdict(list) #store best hyper-parameters for each fold
    inner_loop = True
    if inner_loop:     
        print 'Starting InnerFold computation'        
        clf = innerCVLoop(model_clf,hyper_params,train_X, train_y)
        for hp in hyper_params:
            hp_dict[hp].append(clf.best_estimator_.get_params()[hp])
            
        print 'Ending InnerFold computation'

    else:
        clf = model_clf
        clf.fit(fold_X,fold_y)
    
    if output_type == 'regression':
        #CV_scores    
        r_train = stats.pearsonr(clf.predict(train_X),train_y)
        r_valid = stats.pearsonr(clf.predict(valid_X),valid_y)

        R2_train = clf.score(train_X,train_y) 
        R2_valid = clf.score(valid_X,valid_y)

        MSE_train = mse(clf.predict(train_X),train_y)
        MSE_valid = mse(clf.predict(valid_X),valid_y)

        MAE_train = mae(clf.predict(train_X),train_y)
        MAE_valid = mae(clf.predict(valid_X),valid_y)
        
        return_dict = {'r_train':r_train, 'r_valid':r_valid, 'R2_train':R2_train, 'R2_valid':R2_valid,
            'MSE_train':MSE_train, 'MSE_valid':MSE_valid, 'MAE_train':MAE_train, 'MAE_valid':MAE_valid,
            'hp_dict':hp_dict, 'predicted_fold_score': clf.predict(valid_X), 'actual_fold_scores':valid_y}
        
    elif output_type == 'classify':
        pred_y = clf.predict(valid_X) #Specific label value
        pred_y_prob = clf.predict_proba(valid_X) #Label probabilty (used for ROC)

        #feature imp (issue with the innerCVLoop)
        if model_type == 'RFC':
            feat_imp = clf.best_estimator_.feature_importances_
        elif model_type == 'LR_L1':
            feat_imp = clf.best_estimator_.coef_ 
        else: 
            print "Unknown model for computing feature importance. Setting it to zeros"
            feat_imp = np.zeros(len(pred_y))

        print 'Ending OuterFold computation'

        return_dict = {'actual_fold_scores':valid_y, 'predicted_fold_prob':pred_y_prob, 'predicted_fold_score': pred_y, 
                'hp_dict':hp_dict, 'feat_imp': feat_imp}
    
    else: 
        print 'unknown output_type'

    print 'Ending OuterFold computation'
    
    return return_dict

#Inner Fold Computation (need the imports inside def if you want to parallelize!)
def innerCVLoop(model_clf,hyper_params,fold_X, fold_y):
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Lasso
    from sklearn.svm import SVR
    from sklearn.ensemble import RandomForestRegressor
    from sklearn import grid_search
    clf = grid_search.GridSearchCV(model_clf, hyper_params,cv=3,verbose=0)
    clf.fit(fold_X, fold_y)   
    return clf

def autolabel(rects,_ax):
    for rect in rects:
        height = rect.get_height()
        _ax.text(rect.get_x() + rect.get_width()/2., 1.1*height,
                '{:03.2f}'.format(height),
                ha='center',            # vertical alignment
                va='bottom',             # horizontal alignment
                fontsize = 35)

## Stage 1

In [3]:
# Data imports
my_name = 'Exp6_MCI_demograph'
cohort = 'ADNI2'
clinical_scale = 'ADAS13'
exp_name = '{}_{}_{}'.format(my_name,cohort,clinical_scale)

baseline_dir = '/projects/nikhil/ADNI_prediction/input_datasets/longitudinal_trajectories/'
#in_data_file = 'input_csv/Longitduinal_HC_CT_data.csv'
in_data_file = 'input_csv/Longitduinal_HC_CT_data_MCI_traj.csv'
in_data = pd.read_csv(baseline_dir + in_data_file)

in_data.replace(['Male','Female'],[0,1], inplace=True)

In [159]:
# features / variables
clinical_scale = ['ADAS13']
hc_feat = ['L_CA1','L_subiculum','L_CA4DG','L_CA2CA3','L_stratum','L_Alv','L_Fimb','L_Fornix','L_Mam',
                 'R_CA1','R_subiculum','R_CA4DG','R_CA2CA3','R_stratum','R_Alv','R_Fimb','R_Fornix','R_Mam']
ct_feat = ['AAL_1', 'AAL_2', 'AAL_3','AAL_4', 'AAL_5', 'AAL_6', 'AAL_7', 'AAL_8', 'AAL_9', 'AAL_10',
                 'AAL_11', 'AAL_12', 'AAL_13', 'AAL_14', 'AAL_15', 'AAL_16', 'AAL_17', 'AAL_18', 'AAL_19', 'AAL_20', 
                 'AAL_21', 'AAL_22', 'AAL_23', 'AAL_24', 'AAL_25', 'AAL_26', 'AAL_27', 'AAL_28', 'AAL_29', 'AAL_30', 
                 'AAL_31', 'AAL_32', 'AAL_33', 'AAL_34', 'AAL_35', 'AAL_36', 'AAL_39', 'AAL_40', 'AAL_43', 'AAL_44', 
                 'AAL_45', 'AAL_46', 'AAL_47', 'AAL_48', 'AAL_49', 'AAL_50', 'AAL_51', 'AAL_52', 'AAL_53', 'AAL_54',
                 'AAL_55', 'AAL_56', 'AAL_57', 'AAL_58', 'AAL_59', 'AAL_60', 'AAL_61', 'AAL_62', 'AAL_63', 'AAL_64', 
                 'AAL_65', 'AAL_66', 'AAL_67', 'AAL_68', 'AAL_69', 'AAL_70', 'AAL_79', 'AAL_80', 'AAL_81', 'AAL_82',
                 'AAL_83', 'AAL_84', 'AAL_85', 'AAL_86', 'AAL_87', 'AAL_88', 'AAL_89', 'AAL_90']


# Pick input features (modalities)
mr_feat = hc_feat + ct_feat + clinical_scale

#Pick timepoints for input and output variables
timepoint_mr = '_bl'
timepoint_cs = '_bl'

feat=[]
for c, cs in enumerate(clinical_scale):
    clinical_scale[c] = cs + timepoint_cs
for m, mr in enumerate(mr_feat):
    feat.append(mr + timepoint_mr)

#additional timpoint for structural features (hc or ct)
# timepoint_mr = '_m06'
# for m, mr in enumerate(mr_feat):
#     feat.append(mr + timepoint_mr)
    
#add demographics
feat.append('AGE')
feat.append('APOE4')
feat.append('PTGENDER')


In [160]:
#Trajectory stats
traj_threshold = 'Traj'
X = np.array(in_data[feat])
y = np.squeeze(np.array(in_data[traj_threshold]))
#y_dx = np.squeeze(np.array(in_data[['DX_cat'+timepoint_mr]]))
y_dx = np.squeeze(np.array(in_data[traj_threshold]))

filtered_idx = np.isfinite(y)
y = y[filtered_idx]
y_dx = y_dx[filtered_idx]
X = X[filtered_idx,:]

print 'X shape:{}, y shape:{}, dx shape:{}'.format(X.shape, y.shape, y_dx.shape)
print '# of Class 1 subjects: {}:'.format(len(y[y==1]))

X shape:(156, 100), y shape:(156,), dx shape:(156,)
# of Class 1 subjects: 81:


In [161]:
# Create folds for CV (default Stratified based on DX)
save_experimental_setup = False #Saves X, y, and KF

n_folds = 10
KF_type = 'stratified' #baseline_dir + 'exp_data/CV_Exp6_ADNI1and2_ADAS13.pkl'  #'path/to/previous/setup' or 'stratified' or 'none'
if KF_type == 'none':
    print 'KFold'
    kf = KFold(len(y), n_folds=10)
elif KF_type == 'stratified': 
    print 'Stratified KFold'
    kf = StratifiedKFold(y_dx, n_folds=10)
else:
    print 'Loading previously saved KF from: {}'.format(KF_type)
    old_exp_setup = pickle.load( open(KF_type, "rb" ) )
    kf = old_exp_setup['kf']
    
for train_index, test_index in kf:    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print 'DX distribution (AD,CN,LMCI): {}'.format((np.sum(y_dx[test_index]==0),np.sum(y_dx[test_index]==1),
                                              np.sum(y_dx[test_index]==2)))
    print 'y_train_mean: {}, y_test_mean: {}'.format(np.mean(y_train),np.mean(y_test))

common_keys = in_data[['ImageUID']] #Uniq image ids for mapping subjects

save_path = baseline_dir + 'exp_setup/CV_{}_MR{}_CS{}_hclust_{}.pkl'.format(exp_name,timepoint_mr,timepoint_cs,traj_threshold)    
print 'Saving exp_setup to: {}'.format(save_path)
if save_experimental_setup:    
    exp_setup = {'X': X, 'y': y, 'y_dx': y_dx, 'kf':kf,'common_subs':common_keys,'exp_name':exp_name}    
    pickleIt(exp_setup, save_path)        

Stratified KFold
DX distribution (AD,CN,LMCI): (8, 9, 0)
y_train_mean: 0.517985611511, y_test_mean: 0.529411764706
DX distribution (AD,CN,LMCI): (8, 8, 0)
y_train_mean: 0.521428571429, y_test_mean: 0.5
DX distribution (AD,CN,LMCI): (8, 8, 0)
y_train_mean: 0.521428571429, y_test_mean: 0.5
DX distribution (AD,CN,LMCI): (8, 8, 0)
y_train_mean: 0.521428571429, y_test_mean: 0.5
DX distribution (AD,CN,LMCI): (8, 8, 0)
y_train_mean: 0.521428571429, y_test_mean: 0.5
DX distribution (AD,CN,LMCI): (7, 8, 0)
y_train_mean: 0.517730496454, y_test_mean: 0.533333333333
DX distribution (AD,CN,LMCI): (7, 8, 0)
y_train_mean: 0.517730496454, y_test_mean: 0.533333333333
DX distribution (AD,CN,LMCI): (7, 8, 0)
y_train_mean: 0.517730496454, y_test_mean: 0.533333333333
DX distribution (AD,CN,LMCI): (7, 8, 0)
y_train_mean: 0.517730496454, y_test_mean: 0.533333333333
DX distribution (AD,CN,LMCI): (7, 8, 0)
y_train_mean: 0.517730496454, y_test_mean: 0.533333333333
Saving exp_setup to: /projects/nikhil/ADNI_pred

In [11]:
# QC plots
qc_dict = {}
foldx = 0
idx = 0
for train_index, test_index in kf:        
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
        
    for i in range(X_train.shape[1]): 
        if i < 18:
            modality = 'hc'
        else: 
            modality = 'ct'
            
        vol_mean_train = np.mean(X_train[:,i])
        corr_train = stats.pearsonr(X_train[:,i],y_train)[0]                
        qc_dict[idx]={'structure':mr_feat[i],'subset':'train','vol':vol_mean_train,'r':corr_train,'fold':foldx,'modality':modality}
        idx += 1
        
        vol_mean_test = np.mean(X_test[:,i])
        corr_test = stats.pearsonr(X_test[:,i],y_test)[0]        
        qc_dict[idx]={'structure':mr_feat[i],'subset':'test','vol':vol_mean_test,'r':corr_test,'fold':foldx,'modality':modality}
        idx += 1
    
    foldx += 1
    
qc_df = pd.DataFrame.from_dict(qc_dict,orient='index')
sns.set_context("notebook", font_scale=1)
sns.set_style("ticks")
modality = 'hc'
measure = 'r'
g = sns.factorplot(x='structure', y=measure, hue='subset', col='fold', col_wrap=2, data=qc_df[qc_df.modality==modality], size=5, kind='bar')

In [51]:
a = range(99)
b=a[-2:]
print b


[97, 98]


# Stage 2

In [4]:
# Configs for K-fold validations (nested)
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import grid_search
import datetime
import time
import collections
from scipy.stats import mode
from sklearn.metrics import mean_squared_error as mse
from functools import partial #Parallelize!!! 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from scipy import interp
from sklearn.metrics import accuracy_score as acc
import ipyparallel as ipp
from sklearn import preprocessing
from sklearn.cross_validation import KFold
import pickle
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN #For treating class imbalance

/projects/nikhil/ADNI_prediction/code/conda_envs/adni-conda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [5]:
#Pick model with its configs/hyper-paramsd
def getModel(model_choice,output_type): 
    if output_type == 'regression' :
        if model_choice == 'LR_L1':
            model_clf = Lasso()
            hyper_params = {'alpha':[0.1, 0.05, 0.01]} 
            scale_data = True #Scales HC and CT features    
            feat_imp = False   

        elif model_choice == 'SVR':
            model_clf = SVR()
            hyper_params = {'kernel':['linear','rbf'], 'C':[1,10,25]}
            scale_data = True #Scales HC and CT features        
            feat_imp = False

        elif model_choice == 'RFR':
            model_clf = RandomForestRegressor(n_jobs=4)
            hyper_params = {'n_estimators':[10,50,100,200],'min_samples_split':[2,4,8]}    
            scale_data = False    
            feat_imp = False   
        else:
            print "Unknown model choice"
    
    elif output_type =='classify':
        if model_choice == 'LR_L1':
            model_clf = LogisticRegression(penalty='l1',n_jobs=4,class_weight='balanced')
            hyper_params = {'C':[.1,1,10]} 
            scale_data = True #Scales features (z-score)            
            feat_imp = False #The learned weights of each variable

        elif model_choice == 'SVC':
            model_clf = SVC(class_weight='balanced',probability=True)
            hyper_params = {'kernel':['linear','rbf'], 'C':[.1,1,5,10]}
            scale_data = True #Scales HC and CT features            
            feat_imp = False

        elif model_choice == 'RFC':    
            model_clf = RandomForestClassifier(n_jobs=8,class_weight='balanced' )
            hyper_params = {'n_estimators':[10,50,100,200],'min_samples_split':[2,4,8]}    
            scale_data = False            
            feat_imp = True   #RF oob based feature importance

        else:
            print "Unknown model choice"
    
    else: 
        print 'Unknown output type'

    return {'model_clf':model_clf,'hyper_params':hyper_params,'scale_data':scale_data,'feat_imp':feat_imp}

In [ ]:
# Train and Test models
from functools import partial #Parallelize!!! 

# Load saved experimental setup
exp_name = 'Exp6_MCI_demograph'
Clinical_Scale = 'ADAS13'
timepoint_mr = 'bl'
timepoint_cs = 'bl'
resample_train_data = False
sampling_tech = 'smote'

exp_setup_path = baseline_dir + 'exp_setup/CV_{}_ADNI2_{}_MR_{}_CS_{}_hclust_Traj.pkl'.format(exp_name,Clinical_Scale,timepoint_mr,timepoint_cs)
exp_setup = pickle.load( open(exp_setup_path, "rb" ) )

# Parallization configs for ipython notebook cluster    
rc = ipp.Client()
dview = rc[:]
dview.push(dict(computeOuterFold = computeOuterFold))
dview.push(dict(innerCVLoop = innerCVLoop))

modalities = ['CS','HC_CT_CS']
X_raw = exp_setup['X']
y = exp_setup['y']
kf = exp_setup['kf']
exp_name = exp_setup['exp_name']

output_type = 'classify' #needs to be either 'classify' or 'regression'
if output_type == 'regression':
    model_list = ['LR_L1', 'SVR', 'RFR']
elif output_type == 'classify':
    model_list = ['LR_L1', 'SVC', 'RFC']
else:
    print 'unknow output type'

save_CV_perf = False 

hc_offset = 18
ct_offset = 96
long_offset = 97 #number of total features in each timepoint (18+78+1)
#######################
use_demograph = True
#######################
idx = 1
df_perf_dict = {}
for modality in modalities:    
    if modality == 'HC_CT_CS_long':
        X_modality = X_raw[:,:2*long_offset]
    elif modality == 'HC_CT_CS':
        X_modality = X_raw[:,:long_offset]
        #X_modality = np.hstack((X_raw[:,:5],X_raw[:,9:14],X_raw[:,18:long_offset])) #exclude white matter
    elif modality == 'HC_CT':
        X_modality = X_raw[:,:ct_offset]
        #X_modality = np.hstack((X_raw[:,:5],X_raw[:,9:14],X_raw[:,18:ct_offset])) #exclude white matter
    elif modality == 'HC_CT_long':
        X_modality = np.hstack((X_raw[:,:ct_offset],X_raw[:,long_offset:long_offset+ct_offset]))
    elif modality == 'HC_CS':
        X_modality = np.hstack((X_raw[:,:hc_offset],X_raw[:,ct_offset:long_offset]))
    elif modality == 'CT_CS':
        X_modality = X_raw[:,hc_offset:long_offset]
    elif modality == 'HC_CS_long':
        X_modality = np.hstack((X_raw[:,:hc_offset],X_raw[:,ct_offset:long_offset],X_raw[:,long_offset:long_offset+hc_offset],X_raw[:,long_offset+ct_offset:2*long_offset]))
    elif modality == 'CT_CS_long':
        X_modality = np.hstack((X_raw[:,hc_offset:97],X_raw[:,long_offset+hc_offset:long_offset+97]))
    elif modality == 'HC':
        X_modality = X_raw[:,:hc_offset]
    elif modality == 'CT':
        X_modality = X_raw[:,hc_offset:ct_offset]
    elif modality == 'CS':
        X_modality = X_raw[:,ct_offset:long_offset]
    elif modality == 'CS_long':
        X_modality = np.hstack((X_raw[:,ct_offset:long_offset],X_raw[:,long_offset+ct_offset:2*long_offset]))
    else:
        print "Wrong modality selected..."
    
    if modality in ['HC','CT','HC_CT']:
        use_demograph = False
    if use_demograph:
        X_modality = np.hstack((X_modality,X_raw[:,-2:])) #last two variable are age and apoe4

    ####################
    ## Select a model ##
    ####################       
    for model_choice in model_list:
        model_def = getModel(model_choice,output_type)
        model_clf = model_def['model_clf']
        hyper_params = model_def['hyper_params']
        scale_data = model_def['scale_data']    
        feat_imp = model_def['feat_imp']

        if scale_data:
            X = preprocessing.scale(X_modality)
        else:
            X = X_modality

        print 'Modality: {}, model: {}, X shape {}'.format(modality, model_choice, X.shape)
        #Some paths to store models and performance stats
        CV_model_dir = baseline_dir + 'output/'
        save_model_path = CV_model_dir + exp_name + '_' + model_choice + '_' + modality
        save_model = False 
        
        # Create list of all the fold-subsets (needed for parallelization)
        X_train = []
        X_valid = []
        y_train = []
        y_valid = []    
        for train, valid in kf:
            if resample_train_data:             
                #resample with tools from: https://github.com/scikit-learn-contrib/imbalanced-learn
                if sampling_tech == 'adasyn':                    
                    resamp_tech = ADASYN()
                elif sampling_tech == 'smote':
                    resamp_tech = SMOTE(kind='regular')
                else:
                    resamp_tech = RandomOverSampler()
                
                print 'using {} oversampling technique'.format(sampling_tech)                    
                X_resampled, y_resampled = resamp_tech.fit_sample(X[train], y[train])
                #print 'Original shapes: X{}, y:{}, y==1: {}'.format(X[train].shape, y[train].shape, np.sum(y[train]))
                #print 'Resampled shapes: X{}, y:{}, y==1: {}'.format(X_resampled.shape, y_resampled.shape, np.sum(y_resampled))                
                X_train.append(X_resampled)
                y_train.append(y_resampled)
            else:
                X_train.append(X[train])
                y_train.append(y[train])
            
            # No need to resample test data
            X_valid.append(X[valid])            
            y_valid.append(y[valid])

        # Compute various statistics on the predicted results acorss 10 Folds    
        if output_type == 'regression':
            CV_r_train=[] #pearson r score for each outer fold on train set
            CV_r_valid=[] #pearson r score for each outer fold on validation set
            CV_R2_train=[] #R2 score for each outer fold on train set
            CV_R2_valid=[] #R2 score for each outer fold on validation set
            CV_MSE_train=[] #MSE for each outer fold on train set
            CV_MSE_valid=[] #MSE for each outer fold on validation set    
            CV_MAE_train=[] #MSE for each outer fold on train set
            CV_MAE_valid=[] #MSE for each outer fold on validation set
        elif output_type == 'classify':        
            CV_Acc_valid=[] #Acuracy
            CV_CM_valid=[] #Confusion matrix
            CV_AUC_valid=[] #ROC and AUC                   
            CV_feat_imp = []
        else: 
            print 'unknown output type'

        predicted_CV_scores = []
        actual_CV_scores = []

        #Parallelize k-folds
        mapfunc = partial(computeOuterFold, model_clf=model_clf, hyper_params=hyper_params, output_type=output_type, model_type=model_choice)
        parallel_result = dview.map_sync(mapfunc, X_train, y_train, X_valid, y_valid)    

        hp_dict = collections.defaultdict(list)
        for p, pr in enumerate(parallel_result):
            if output_type == 'regression':
                CV_r_train.append(pr['r_train'])
                CV_r_valid.append(pr['r_valid'])
                CV_R2_train.append(pr['R2_train'])
                CV_R2_valid.append(pr['R2_valid'])
                CV_MSE_train.append(pr['MSE_train'])
                CV_MSE_valid.append(pr['MSE_valid'])
                CV_MAE_train.append(pr['MAE_train'])
                CV_MAE_valid.append(pr['MAE_valid'])
                predicted_CV_scores.append(pr['predicted_fold_score'])
                actual_CV_scores.append(pr['actual_fold_scores'])                
                    
            elif output_type == 'classify':
                y_pred_score = pr['predicted_fold_score']
                y_pred_prob = pr['predicted_fold_prob']
                y_act = pr['actual_fold_scores']
                predicted_CV_scores.append(y_pred_score)
                actual_CV_scores.append(y_act)        

                #Acc    
                CV_accuracy = acc(y_pred_score,y_act)
                CV_Acc_valid.append(CV_accuracy)                

                #AUC                
                CV_auc = roc_auc_score(y_act, y_pred_prob[:,1])
                CV_AUC_valid.append(CV_auc)

                #confusion matrix
                cm = confusion_matrix(y_pred_score,y_act)
                cm_normalized = cm.astype('float')/cm.sum(axis=1)[:, np.newaxis]
                CV_CM_valid.append(cm_normalized)
                
                df_perf_dict[idx] = {'modality':modality,'model_choice':model_choice,'KF':p+1, 'act_scores':y_act,
                                    'pred_scores':y_pred_score,'pred_prob':y_pred_prob, 'Accuracy':CV_accuracy,
                                    'AUC':CV_auc,'CM':cm_normalized}
                idx+=1
                
            for hp in hyper_params:
                hp_dict[hp].append(pr['hp_dict'][hp])
                
        #Find out most frequent hyper-params during cross-val    
        hp_mode = {}
        for hp in hyper_params:
            hp_mode[hp] = mode(hp_dict[hp])[0][0]

        if output_type == 'regression':
            print 'most frequent hp:' + str(hp_mode)
            print 'CV r (mean, median, std_err): ' + '{:04.2f},{:04.2f},{:04.2f}'.format(np.mean(zip(*CV_r_valid)[0]),np.median(zip(*CV_r_valid)[0]),stats.sem(zip(*CV_r_valid)[0]))
            print 'CV R2 (mean, median, std_err): ' + '{:04.2f},{:04.2f}, {:04.2f}'.format(np.mean(CV_R2_valid),np.median(CV_R2_valid),stats.sem(CV_R2_valid))
            print 'CV MSE (mean, median, std_err): ' + '{:04.2f},{:04.2f}, {:04.2f}'.format(np.mean(CV_MSE_valid),np.median(CV_MSE_valid),stats.sem(CV_MSE_valid))
            print 'CV MAE (mean, median, std_err): ' + '{:04.2f},{:04.2f}, {:04.2f}'.format(np.mean(CV_MAE_valid),np.median(CV_MAE_valid),stats.sem(CV_MAE_valid))
            print ''
            
        elif output_type == 'classify':
            print 'most frequent hyper-params:' + str(hp_mode)
            print 'CV Acc (mean, std_err): {:04.2f}, {:04.2f}'.format(np.mean(CV_Acc_valid),stats.sem(CV_Acc_valid))
            print 'CV AUC (mean, std_err): {:04.2f}, {:04.2f}'.format(np.mean(CV_AUC_valid),stats.sem(CV_AUC_valid))
            print 'CV Confusion Matrix (mean): {}'.format(np.mean(np.array(CV_CM_valid),axis=0))
            print ''
            plot_name = 'ROC'
             
#Save df style dictionaly for seaborn plots
# df_perf_dict_path = baseline_dir + 'df_perf_dict_{}_{}.pkl'.format(exp_name, Clinical_Scale)                
# pickleIt(df_perf_dict,df_perf_dict_path)
# print 'saving results at: {}'.format(df_perf_dict_path)

Modality: CS, model: LR_L1, X shape (156, 3)
most frequent hyper-params:{'C': array([10])}
CV Acc (mean, std_err): 0.59, 0.04
CV AUC (mean, std_err): 0.58, 0.05
CV Confusion Matrix (mean): [[ 0.57587302  0.42412698]
 [ 0.39164502  0.60835498]]

Modality: CS, model: SVC, X shape (156, 3)
most frequent hyper-params:{'kernel': array(['linear'], 
      dtype='|S6'), 'C': array([ 0.1])}
CV Acc (mean, std_err): 0.60, 0.05
CV AUC (mean, std_err): 0.57, 0.05
CV Confusion Matrix (mean): [[ 0.56865079  0.43134921]
 [ 0.38378788  0.61621212]]

Modality: CS, model: RFC, X shape (156, 3)
most frequent hyper-params:{'n_estimators': array([10]), 'min_samples_split': array([2])}
CV Acc (mean, std_err): 0.55, 0.03
CV AUC (mean, std_err): 0.58, 0.04
CV Confusion Matrix (mean): [[ 0.53273504  0.46726496]
 [ 0.42358586  0.57641414]]

Modality: HC_CT_CS, model: LR_L1, X shape (156, 99)
most frequent hyper-params:{'C': array([1])}
CV Acc (mean, std_err): 0.58, 0.04
CV AUC (mean, std_err): 0.65, 0.04
CV Conf

/projects/nikhil/ADNI_prediction/code/conda_envs/adni-conda/lib/python2.7/site-packages/scipy/stats/stats.py:250: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [ ]:
#ROC curves

#Load perf_data
exp_perf_path = baseline_dir + 'df_perf_dict_{}_{}.pkl'.format(exp_name, Clinical_Scale)  
exp_perf = pickle.load( open(exp_perf_path, "rb" ) )
df_perf = pd.DataFrame.from_dict(exp_perf,orient='index')

marker_list = ['o','d','s','<','^','>','D','*','H']
plt.style.use('seaborn-white')
plt.rcParams['figure.figsize'] = (50, 50)

mk=0
for modality in ['CS','HC_CT_CS']:
    for model_choice in ['LR_L1','SVC','RFC']:
        df = df_perf[(df_perf.model_choice==model_choice)&(df_perf.modality==modality)]
        CV_fpr_tpr = []
        CV_fpr_tpr_thrsh = []
        for y_act, y_pred_prob in zip(df['act_scores'].values,df['pred_prob'].values):
            fpr, tpr, thresholds = roc_curve(y_act, y_pred_prob[:,1]) 
            CV_fpr_tpr.append((fpr,tpr))        
            CV_fpr_tpr_thrsh.append(thresholds)
            
        mean_tpr = 0.0
        mean_fpr = np.linspace(0, 1, 100)
        all_tpr = []

        for i, (fpr, tpr) in enumerate(CV_fpr_tpr):
            mean_tpr += interp(mean_fpr, fpr, tpr)
            mean_tpr[0] = 0.0
            roc_auc = auc(fpr, tpr)

        mean_tpr /= len(CV_fpr_tpr)
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)
        
        if modality == 'CS':
            fig_modality = 'CL Var'
        elif modality == 'HC_CT_CS':
            fig_modality = 'CL + MR Var'
            
        with sns.axes_style("whitegrid"):
            label_str = '{}, {}, mean ROC (area = {:3.2f})'.format(fig_modality, model_choice, mean_auc)
            plt.plot(mean_fpr, mean_tpr, label=label_str, lw=4, marker = marker_list[mk],markersize=20)                    
            mk+=1

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate',fontsize = 50)
plt.ylabel('True Positive Rate',fontsize = 50)
plt.title('Receiver operating characteristic',fontsize = 50)
plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck', lw=4)
plt.legend(loc="lower right",fontsize = 50)
plt.xticks(fontsize = 50)
plt.yticks(fontsize = 50)

save_figure = True
fig_name = 'HBM2017_fig2.png'
if save_figure:    
    plt.savefig(baseline_dir + fig_name, dpi=1000) 
    print 'saving fig at: {}'.format(baseline_dir + fig_name)


In [9]:
baseline_dir + fig_name

'/projects/nikhil/ADNI_prediction/input_datasets/longitudinal_trajectories/HBM2017_fig2.png'

In [153]:
in_data

,Unnamed: 0,PTID,ImageUID,AGE,PTGENDER,APOE4,DX_bl,DX_cat_bl,ADAS13_bl,MMSE_bl,...,AAL_88_m06_diff,AAL_88_m12_diff,AAL_88_m24_diff,AAL_89_m06_diff,AAL_89_m12_diff,AAL_89_m24_diff,AAL_90_m06_diff,AAL_90_m12_diff,AAL_90_m24_diff,Traj
0,0,002_S_4229,260229,66.4,Male,0.0,MCI,1.0,11.0,29.0,...,-0.088440,-0.123422,-0.006649,0.007642,-0.017480,-0.025395,-0.025068,-0.151539,0.104452,0
1,1,002_S_4447,280562,67.5,Female,1.0,MCI,1.0,7.0,27.0,...,-0.332538,-0.367894,0.820834,0.115986,-0.078142,0.169506,-0.261451,-0.222107,0.127221,1
2,2,003_S_4354,278119,75.7,Male,0.0,MCI,1.0,20.0,29.0,...,-0.667623,-0.045083,-0.567945,-0.338529,0.082627,-0.358237,-0.555292,0.027850,-0.677290,1
3,3,005_S_4168,254768,82.2,Male,1.0,MCI,1.0,11.0,29.0,...,-0.113999,-0.211851,-0.131536,-0.080441,-0.058579,-0.042889,-0.160080,-0.163682,-0.204514,1
4,4,005_S_4185,258625,80.4,Male,0.0,MCI,1.0,12.0,28.0,...,-0.047390,0.129344,-0.692548,-0.038740,0.055245,-0.237143,0.013180,0.171727,-0.180387,1
5,5,006_S_4363,270050,73.5,Female,0.0,MCI,1.0,21.0,26.0,...,-0.148827,-0.057952,0.130562,-0.021139,-0.019037,-0.023215,-0.139353,-0.053405,0.182538,0
6,6,006_S_4679,312648,69.1,Male,1.0,MCI,1.0,19.0,28.0,...,-0.822328,-2.311256,-0.869241,-0.469665,-2.089346,-0.720852,-0.390735,-2.259789,-0.248947,0
7,7,006_S_4713,306760,70.5,Male,1.0,MCI,1.0,14.0,30.0,...,-0.040295,-1.024037,0.075535,0.167552,-0.610306,-0.027262,0.026069,-0.571623,-0.002743,1
8,8,007_S_4467,281397,61.9,Female,2.0,MCI,1.0,18.0,27.0,...,-0.168755,-0.070963,-0.215508,0.056800,-0.163394,0.059593,-0.007527,0.140176,-0.058728,1
9,9,007_S_4611,295877,66.9,Male,1.0,MCI,1.0,11.0,25.0,...,0.020921,-0.150246,-0.245842,-0.102141,0.055866,-0.122299,0.002422,-0.110934,-0.149667,1
